In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib.cm import rainbow
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import cross_val_score
import pickle


In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
X_train=pd.read_csv(io.BytesIO(uploaded['train.csv']))
y_train=pd.read_csv(io.BytesIO(uploaded['target.csv']))
X_test=pd.read_csv(io.BytesIO(uploaded['test.csv']))


In [ ]:
y_train.info()

In [ ]:
def get_dummies_for_col(dataframe,dummy_col, threshold=0.1):

    # removes the bind
    dummy_col = dummy_col.copy()

    # what is the ratio of a dummy in whole column
    count = dummy_col.value_counts()/dummy_col.value_counts().sum()
    
    # cond whether the ratios is higher than the threshold
    mask = dummy_col.isin(count[count > threshold].index)

    # replace the ones which ratio is lower than the threshold by a special name
    dummy_col[~mask] = "others"
    
    #dataset.drop(dummy_col.name, axis=1, inplace=True)
    dataframe.drop(dummy_col.name, axis=1, inplace=True)
    
    new_df=pd.get_dummies(dummy_col, prefix=dummy_col.name)
    
    dataframe=dataframe.join(new_df)

    return dataframe

In [ ]:
def preprocessMyFields(dataset):
    dataset.drop('amount_tsh', axis=1, inplace=True)
    dataset.drop('num_private', axis=1, inplace=True)
    dataset.drop('wpt_name', axis=1, inplace=True)
    dataset.drop('subvillage', axis=1, inplace=True)
    dataset.drop('population', axis=1, inplace=True)
    dataset.drop('region_code', axis=1, inplace=True)
    dataset.drop('lga', axis=1, inplace=True)
    dataset.drop('region', axis=1, inplace=True)
    dataset.drop('ward', axis=1, inplace=True)
    dataset.drop('longitude', axis=1, inplace=True)
    dataset.drop('latitude', axis=1, inplace=True)
    dataset.drop('recorded_by', axis=1, inplace=True)
    dataset.drop('scheme_name', axis=1, inplace=True)
    dataset.drop('waterpoint_type_group', axis=1, inplace=True)
    dataset.drop('source_type', axis=1, inplace=True)
    dataset.drop('source', axis=1, inplace=True)
    dataset.drop('quantity', axis=1, inplace=True)
    dataset.drop('quality_group', axis=1, inplace=True)
    dataset.drop('payment', axis=1, inplace=True)
    dataset.drop('management', axis=1, inplace=True)
    dataset.drop('scheme_management', axis=1, inplace=True)
    dataset.drop('extraction_type_group', axis=1, inplace=True)
    dataset.drop('extraction_type_class', axis=1, inplace=True)
    dataset.drop('date_recorded', axis=1, inplace=True)
    dataset.drop('district_code', axis=1, inplace=True)

    #dataset['population']=dataset['population'].replace({0:dataset['population'].mean() })
    dataset['construction_year']=dataset['construction_year'].replace({0:dataset['construction_year'].quantile(0.5) })
    dataset['public_meeting']=dataset['public_meeting'].replace({True: 1, False: 0})
    dataset['permit']=dataset['permit'].replace({True: 1, False: 0})
    max_publicmeeting=dataset['public_meeting'].value_counts().idxmax()
    max_permit=dataset['permit'].value_counts().idxmax()

    dataset['public_meeting']=dataset['public_meeting'].fillna(max_publicmeeting)
    dataset['permit']=dataset['permit'].fillna(max_permit)
    max_funder=dataset['funder'].value_counts().idxmax()
    dataset['funder']=dataset['funder'].fillna(max_funder)
    dataset['gps_height']=dataset['gps_height'].replace({0:dataset['gps_height'].mean() })
    max_installer = dataset['installer'].value_counts().idxmax()
    dataset['installer']=dataset['installer'].fillna(max_installer)
    
    
    dataset=get_dummies_for_col(dataset,dataset['funder'])
    dataset=get_dummies_for_col(dataset,dataset['installer'])
    dataset=get_dummies_for_col(dataset,dataset['basin'])
    dataset=get_dummies_for_col(dataset,dataset['management_group'])
    dataset=get_dummies_for_col(dataset,dataset['extraction_type'],0.13)
    dataset=get_dummies_for_col(dataset,dataset['payment_type'])
    dataset=get_dummies_for_col(dataset,dataset['water_quality'],0.01)
    dataset=get_dummies_for_col(dataset,dataset['quantity_group'])
    dataset=get_dummies_for_col(dataset,dataset['source_class'])
    dataset=get_dummies_for_col(dataset,dataset['waterpoint_type'],0.2)
    
    
    
    #dataset.drop('id', axis=1, inplace=True)
    return dataset
                

In [ ]:
X_train_preprocessed = preprocessMyFields(X_train)
X_test_preprocessed = preprocessMyFields(X_test)

In [ ]:
X_train_preprocessed.info()

In [ ]:
scaler = StandardScaler().fit(X_train_preprocessed) 
scaled_dataset_X = scaler.transform(X_train_preprocessed) 
scaled_testset_X=scaler.transform(X_test_preprocessed)
#y_train=pd.get_dummies(y_train)


In [ ]:
# Fit the model on training set
model = RandomForestClassifier(n_estimators=100)
#print(cross_val_score(model, scaled_dataset_X, y_train, cv=10, scoring='accuracy').mean())


In [ ]:

model.fit(scaled_dataset_X, y_train)
# save the model to disk
#filename = 'finalized_model.sav'
#pickle.dump(model, open(filename, 'wb'))
 

In [ ]:
# some time later...
 
# load the model from disk
#loaded_model = pickle.load(open(filename, 'rb'))
prediction_test = model.predict(scaled_testset_X)

submission_df = pd.DataFrame({"id":X_test["id"].values,"status_group":prediction_test})
submission_df.to_csv(io.BytesIO(uploaded['submissions.csv']),index=False)